In [284]:
# 重回帰分析

# 目的変数yを説明変数x1, x2, x3,... ,xdの線形関数と定数項で説明するモデル
# y = B0 + B1*x1 + ... + Bd*xd + ε
# ただし、B1,B2,..Bd∈Rは各変数のyへの影響を表現する「回帰係数」で、B0は切片項にあたる
# また、εは誤差項で平均0かつ分散σ**2とする

# 具体例：wineデータセットで実践
import pandas as pd
from sklearn.datasets import load_wine
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

wine_data = load_wine()
wine_df = pd.DataFrame(wine_data.data, columns=wine_data.feature_names)

wine_df_train, wine_df_test = train_test_split(wine_df, train_size=150)

x_train = wine_df_train.drop('alcohol', axis=1) # 説明変数x1, x2, x3,... ,xd
y_train = wine_df_train['alcohol'] # 目的変数y

x_test = wine_df_test.drop('alcohol', axis=1)
y_test = wine_df_test['alcohol']

model = LinearRegression()
model.fit(x_train, y_train)
pred_y = model.predict(x_test)

print(f"予測結果：{[round(pred_y[n], 2) for n in range(len(pred_y))]}")
print(f"実際の値：{y_test.to_list()}")


予測結果：[14.08, 12.01, 13.55, 13.31, 13.35, 13.38, 13.74, 13.8, 12.71, 12.99, 13.39, 12.39, 13.1, 12.47, 13.54, 12.43, 13.01, 13.31, 13.8, 12.2, 13.81, 12.33, 12.68, 13.47, 13.43, 13.77, 12.52, 12.33]
実際の値：[13.82, 12.69, 14.34, 14.06, 13.17, 14.02, 14.38, 14.39, 12.37, 13.24, 13.3, 12.17, 13.36, 11.61, 13.83, 12.37, 13.5, 12.77, 14.06, 12.0, 13.75, 11.65, 11.62, 13.9, 13.56, 12.82, 11.87, 13.49]


In [285]:
# 目的変数の変動を説明変数で説明できる部分とそれ以外の部分に分けることで、
# モデルの説明力を調べることができる。「決定係数」R**2はこの考えを反映させたものである
# 決定係数が大きければそれだけデータのへの当てはまりが良いことを意味する

from sklearn.metrics import r2_score

print(f"決定係数(r2):{round(r2_score(y_test, pred_y),3)}")

# なお、決定係数は変数を増やせば増やすほど増大するので、変数の数について調整した「自由度調整済み決定係数」を用いることが多い

def adj_r2_score(y_true, y_pred, n_dim):
    y_true_list = y_true.tolist()
    y_pred_list = y_pred.tolist()
    data_size = len(y_true)
    ysos = 0
    rsos = 0
    y_mean = y_true.mean()
    for i in range(data_size):
        ysos += (y_mean-y_true_list[i])**2
        residual = y_true_list[i] - y_pred_list[i]
        rsos += residual**2
    
    return 1-(rsos/(data_size-n_dim-1))/(ysos/(data_size-1))

print(f"自由度調整済み決定係数(r2):{round(adj_r2_score(y_test, pred_y, len(x_test.columns)),3)}")

決定係数(r2):0.579
自由度調整済み決定係数(r2):0.242


In [286]:
# 重回帰分析の検定

# 重回帰分析においては、変数の「有意性検定」が重要である。
# たとえば、回帰変数Bkがゼロか非ゼロかを検定することにより、変数xkがyに有意に影響しているかどうかがわかる
# 変数の有意性検定を一般化すると、以下のように記述できる
# あるq<d+1を満たすqに対し、行列A∈R**(q*(d+1))はランクがrank(A)=qかつ像がIm(A**T)⊂Im(X**T)を満たすとする
# このAを用いて、帰無仮説H0：A*B=0、対立仮説H1：A*B≠0 として書ける仮説検定を考える
# たとえば、A=(0,0,...,1,0,..0)のようにk+1番目の成分のみが1のベクトルを考えれば、A*B=0⇔Bk=0となる
# このとき、Θ0={B∈R**(d+1) | A*B=0}として、
# R0**2 = min||Y-X*B||**2 (B∈Θ0) とおき
# R1**2 = min||Y-X*B||**2 (B∈R*(d+1)) とおく
# R0**2は帰無仮説のもとで二乗損失がどれだけ小さくできるかを表し、
# R1**2は対立仮説のもとで二乗損失がどれだけ小さくできるかを表す。なおR1**2=||e||**2である
# もし、Ro**2とR1**2が大きく変わる場合は、帰無仮説は誤っているとして棄却すればよい
# そこで統計検定量として、T = ((Ro**2-R1**2)/q)/(R1**2/(n-d-1)) を考える
# すると帰無仮説のもと、T~F(q, n-d-1) が成り立つ。なおTは統計量である

In [287]:
# 回帰係数Bk=0(k∈{1,..,d})の検定
# A=(0,0,...,1,0,..0)(k+1番目の成分のみが1)とする

import statsmodels.api as sm
import numpy as np

model = sm.OLS(y_test, sm.add_constant(x_test))
result = model.fit()
A = np.identity(len(result.params))
A = A[1:,:]
print(result.f_test(A))

<F test: F=4.710884220321564, p=0.003004107554451902, df_denom=15, df_num=12>


In [288]:
# 定数項B0=0の検定
# A=(1,0,...,0,0,..0)(1番目の成分のみが1)とする

A = np.identity(len(result.params))
A = A[0,:]
print(result.f_test(A))

<F test: F=42.45762290384734, p=9.756380467303627e-06, df_denom=15, df_num=1>


In [289]:
# 正則化

# 重回帰分析ではX**T*Xが可逆であることを仮定していたが、高次元データを扱う場合は必ずしもそうなるとは限らない
# そこでX**T*Xの条件数が悪い場合でも安定した推定を行うために「正則化」が有用である
# 代表的な正則化手法としては、L1正則化とL2正則化がある
# L1正則化を用いた重回帰はLasso回帰、L2正則化を用いた重回帰はRidge回帰ともよばれる
# 正則化を加えることで推定の分散を抑えることができる。
# そのため、「学習データ」に合わせすぎて予測精度が悪くなる「過適合(過学習)」を抑止することができる
# 一方で正則化が強すぎると推定量が縮小しすぎて、予測精度が悪化する「過小適合」が発生する点に注意

In [290]:
# L1正則化(Lasso回帰)
# 正則化項としてのL1ノルムを使用。ベクトル成分の絶対値の和(マンハッタン距離と呼ばれる)
# 回帰係数Bのいくつかの成分は0(=「スパース」)になり、凸最適化によって変数選択が可能になる

from sklearn.linear_model import Lasso

# Lasso回帰
model = Lasso(alpha=0.1)

# モデル学習と予測
model.fit(x_train, y_train)
pred_y = model.predict(x_test)

lasso_result_df = pd.DataFrame({
    "variable":x_train.columns,
    "coef":model.coef_
})
print("回帰係数")
print(lasso_result_df.to_string(index=False))

print(f"決定係数(r2):{round(r2_score(y_test, pred_y),3)}")
print(f"自由度調整済み決定係数(r2):{round(adj_r2_score(y_test, pred_y, len(x_test.columns)),3)}")

回帰係数
                    variable      coef
                  malic_acid  0.033921
                         ash  0.000000
           alcalinity_of_ash -0.016918
                   magnesium -0.001197
               total_phenols  0.000000
                  flavanoids  0.000000
        nonflavanoid_phenols -0.000000
             proanthocyanins -0.000000
             color_intensity  0.110089
                         hue -0.000000
od280/od315_of_diluted_wines  0.000000
                     proline  0.001288
決定係数(r2):0.626
自由度調整済み決定係数(r2):0.326


In [291]:
# L2正則化(Ridge回帰)
# 正則化項としてのL2ノルムを使用。ベクトル成分の絶対値の和(ユークリッド距離と呼ばれる)
# 正則化パラメータを洗濯する基準として「Mallows' CP基準」がある

from sklearn.linear_model import Ridge

# Ridge回帰
model = Ridge(alpha=1.0)

# モデル学習と予測
model.fit(x_train, y_train)
pred_y = model.predict(x_test)

ridge_result_df = pd.DataFrame({
    "variable":x_train.columns,
    "coef":model.coef_
})

print("回帰係数")
print(ridge_result_df.to_string(index=False))

print(f"決定係数(r2):{round(r2_score(y_test, pred_y),3)}")
print(f"自由度調整済み決定係数(r2):{round(adj_r2_score(y_test, pred_y, len(x_test.columns)),3)}")

回帰係数
                    variable      coef
                  malic_acid  0.146343
                         ash  0.205658
           alcalinity_of_ash -0.044983
                   magnesium -0.001426
               total_phenols  0.111888
                  flavanoids -0.005925
        nonflavanoid_phenols -0.246038
             proanthocyanins -0.154032
             color_intensity  0.156572
                         hue  0.275040
od280/od315_of_diluted_wines  0.118168
                     proline  0.000914
決定係数(r2):0.595
自由度調整済み決定係数(r2):0.271


In [292]:
# Elastic-Net
# L1正則化とL2正則化を混ぜたもので、 2つの相関が強い変数があるとその2つの変数間で
# 変数選択が安定しないL1正則化の欠点を克服。相関の高い両方の変数を用いる

from sklearn.linear_model import ElasticNet

# Ridge回帰
model = ElasticNet(alpha=1.0, l1_ratio=0.5) # l1_ratioはL1ノルムとL2ノルムの比率

# モデル学習と予測
model.fit(x_train, y_train)
pred_y = model.predict(x_test)

eln_result_df = pd.DataFrame({
    "variable":x_train.columns,
    "coef":model.coef_
})

print("回帰係数")
print(eln_result_df.to_string(index=False))

print(f"決定係数(r2):{round(r2_score(y_test, pred_y),3)}")
print(f"自由度調整済み決定係数(r2):{round(adj_r2_score(y_test, pred_y, len(x_test.columns)),3)}")

回帰係数
                    variable      coef
                  malic_acid  0.000000
                         ash  0.000000
           alcalinity_of_ash -0.000000
                   magnesium -0.000000
               total_phenols -0.000000
                  flavanoids -0.000000
        nonflavanoid_phenols  0.000000
             proanthocyanins -0.000000
             color_intensity  0.020206
                         hue -0.000000
od280/od315_of_diluted_wines -0.000000
                     proline  0.001540
決定係数(r2):0.531
自由度調整済み決定係数(r2):0.156
